![banner.png](banner.png)

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Imports</h2>

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, min, max, count
import os

In [7]:
os.environ["HADOOP_HOME"] = "D:\\hadoop"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["HADOOP_HOME"], "bin")

In [8]:
spark = SparkSession.builder \
    .appName("IMDbProject") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [9]:
# Confirm Spark is running
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.5


In [ ]:
# spark.conf.set("spark.sql.shuffle.partitions", "4")
# spark.conf.set("spark.executor.memory", "2g")

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Load Datasets</h2>

In [3]:
# Paths
IMDB_PATH = '../data/raw/imdb'
TMDB_PATH = '../data/raw/tmdb'
RAW_PARQUET_PATH = '../data/raw_parquet'
PROCESSED_PATH = '../data/processed'

In [10]:
# Test

# Load just ratings file
ratings = spark.read.csv(
    os.path.join(IMDB_PATH, 'title.ratings.tsv'),
    sep='\t',
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

# Quick check the schema (optional)
ratings.printSchema()

# Quick check the data (optional)
ratings.show(5)

# Test writing ratings to Parquet (this is the test)
ratings.write.mode('overwrite').parquet(os.path.join(PROCESSED_PATH, 'ratings.parquet'))

print("✅ Ratings saved to Parquet successfully!")


root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2136|
|tt0000002|          5.5|     289|
|tt0000003|          6.4|    2170|
|tt0000004|          5.3|     185|
|tt0000005|          6.2|    2902|
+---------+-------------+--------+
only showing top 5 rows



Py4JJavaError: An error occurred while calling o39.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputComm

In [4]:
# Make sure output folders exist
os.makedirs(RAW_PARQUET_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

imdb_files = {
    "basics": "title.basics.tsv",
    "akas": "title.akas.tsv",
    "crew": "title.crew.tsv",
    "episode": "title.episode.tsv",
    "principals": "title.principals.tsv",
    "ratings": "title.ratings.tsv",
    "names": "name.basics.tsv"
}

### Load IMDB data to Spark Dataframes

In [5]:
imdb_dfs = {
    key: spark.read.csv(
        os.path.join(IMDB_PATH, filename),
        sep='\t',
        header=True,
        inferSchema=True,
        nullValue='\\N'
    )
    for key, filename in imdb_files.items()
}

for name, df in imdb_dfs.items():
    print(f"IMDb - {name}:")
    df.show(5)

IMDb - basics:
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1

In [5]:
imdb_dfs = {
    key: spark.read.csv(
        os.path.join(IMDB_PATH, filename),
        sep='\t',
        header=True,
        inferSchema=True,
        nullValue='\\N'
    )
    for key, filename in imdb_files.items()
}

for name, df in imdb_dfs.items():
    print(f"IMDb - {name}:")
    df.show(5)

IMDb - basics:
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1

### Save unfiltered IMDB files to Parquet

In [ ]:
# for name, df in imdb_dfs.items():
#     parquet_path = os.path.join(RAW_PARQUET_PATH, f'{name}.parquet')
#     df.write.mode('overwrite').parquet(parquet_path)
#     print(f"Saved {name} to {parquet_path}")

### Load TMDB data to Spark Dataframes

In [6]:
tmdb_path = os.path.join(TMDB_PATH, 'TMDB_movie_dataset_v11.csv')

tmdb_df = spark.read.csv(
    tmdb_path,
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

print("TMDb Data Preview:")
tmdb_df.show(5)

TMDb Data Preview:
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|          title|vote_average|vote_count|  status|release_date|   revenue|runtime|adult|       backdrop_path|   budget|            homepage|  imdb_id|original_language| original_title|            overview|          popularity|         poster_path|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+-------

In [7]:
tmdb_path = os.path.join(TMDB_PATH, 'TMDB_movie_dataset_v11.csv')

tmdb_df = spark.read.csv(
    tmdb_path,
    header=True,
    inferSchema=True,
    nullValue='\\N'
)

print("TMDb Data Preview:")
tmdb_df.show(5)

TMDb Data Preview:
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|          title|vote_average|vote_count|  status|release_date|   revenue|runtime|adult|       backdrop_path|   budget|            homepage|  imdb_id|original_language| original_title|            overview|          popularity|         poster_path|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+------+---------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+---------------+--------------------+-------

### Save TMDB file to Parquet

In [ ]:
# TMDB_PARQUET_PATH = '../data/raw_parquet'
# os.makedirs(TMDB_PARQUET_PATH, exist_ok=True)

# tmdb_df.write.mode('overwrite').parquet(os.path.join(TMDB_PARQUET_PATH, 'TMDB_movie_dataset_v11.parquet'))
# print("Saved TMDb to parquet")

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Exploratory Data Analysis</h2>

### Preliminary EDA

In [8]:
imdb_basics = imdb_dfs['basics']

# Total counts for each dataset
print("Row counts per file:")
for name, df in imdb_dfs.items():
    print(f"{name}: {df.count():,} rows")

# Date range in imdb_basics
date_range = imdb_basics.select(min(col("startYear")).alias("min_year"), max(col("startYear")).alias("max_year"))
date_range.show()

# Top genres
genres_exploded = imdb_basics.withColumn("genre", explode(split(col("genres"), ",")))
top_genres = genres_exploded.groupBy("genre").count().orderBy(col("count").desc())
top_genres.show(10)

Row counts per file:
basics: 11,485,855 rows
akas: 51,496,209 rows
crew: 11,485,855 rows
episode: 8,832,680 rows
principals: 91,154,932 rows
ratings: 1,539,107 rows
names: 14,217,591 rows
+--------+--------+
|min_year|max_year|
+--------+--------+
|    1874|    2031|
+--------+--------+

+-----------+-------+
|      genre|  count|
+-----------+-------+
|      Drama|3237636|
|     Comedy|2236773|
|  Talk-Show|1424798|
|      Short|1228429|
|Documentary|1096234|
|       News|1080456|
|    Romance|1072271|
|     Family| 844672|
| Reality-TV| 643344|
|  Animation| 572468|
+-----------+-------+
only showing top 10 rows



### Loading Using Parquet Files

In [ ]:
# # Load from saved Parquets (raw)
# imdb_dfs = {}
# imdb_files = ["basics", "akas", "crew", "episode", "principals", "ratings", "names"]

# for name in imdb_files:
#     path = os.path.join(RAW_PARQUET_PATH, f'{name}.parquet')
#     imdb_dfs[name] = spark.read.parquet(path)

### Filter to `movies` only

In [7]:
imdb_basics_filtered = imdb_dfs['basics'].filter(col("titleType") == "movie")

# # Save filtered basics immediately to processed folder
# imdb_basics.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'basics.parquet'))

In [8]:
# Filter akas
imdb_akas_filtered = imdb_dfs['akas'] \
    .join(imdb_basics_filtered.select("tconst"), imdb_dfs['akas'].titleId == imdb_basics_filtered.tconst, "inner") \
    .drop("tconst")

# imdb_akas.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'akas.parquet'))

# Filter ratings
imdb_ratings_filtered = imdb_dfs['ratings'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_ratings.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'ratings.parquet'))

# Filter principals
imdb_principals_filtered = imdb_dfs['principals'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_principals.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'principals.parquet'))

# Filter crew
imdb_crew_filtered = imdb_dfs['crew'] \
    .join(imdb_basics_filtered.select("tconst"), "tconst", "inner")

# imdb_crew.write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'crew.parquet'))

# For now, leaving `names` alone since it's a people table not directly tied to titleType.
imdb_names = imdb_dfs['names']
# imdb_dfs['names'].write.mode('overwrite').parquet(os.path.join(PROCESSED_PARQUET_PATH, 'names.parquet'))

# Episodes is usually for TV shows, so we can skip it, or handle if needed later.
imdb_episodes = imdb_dfs['episode']

#### Record Counts

In [ ]:
imdb_basics_filtered.count()
imdb_akas_filtered.count()
imdb_ratings_filtered.count()
imdb_principals_filtered.count()
imdb_crew_filtered.count()

In [ ]:
tmdb_movies.count()

#### Schema Checks

In [ ]:
# Print schemas for IMDb files
for name, df in imdb_dfs.items():
    print(f"Schema for IMDb file: {name}")
    df.printSchema()
    print("="*40)  # Just for cleaner separation

# Print schema for TMDb file
print("Schema for TMDb file:")
tmdb_df.printSchema()

### 1. Highest Rated Movies in 2023

**Balancing Ratings and Number of Votes:**



#### Filter 2023 Data

### 2. Most Popular Actors/Actresses in 2023

### 3. User-Movie Trends

### 4. Metric of a 'Hit Movie'

<h2 style="color:#de4a48; background-color:#fce19a; padding: 10px; text-align:left; border: 1px solid #fce19a;">Predictive Modelling</h2>